In [1]:
import re
import time
from colorama import Fore
import openpyxl
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import random
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import math

In [2]:
options = Options()

options.add_argument('--disable-application-cache')
options.add_argument('--disable-blink-features=AutomationControlled')
# options.add_argument("--disable-cookies")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.add_argument("start-maximized")
options.add_argument('--disable-gpu')
# options.add_argument("--headless")  
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument('--ignore-urlfetcher-cert-requests')
options.add_argument('--no-first-run')
options.add_argument("--disable-popup-blocking") 


driver = webdriver.Chrome(options=options)

In [3]:
link='https://annuairesante.ameli.fr/trouver-un-professionnel-de-sante/ophtalmologiste'

In [4]:
driver.get(link)

In [5]:
time.sleep(2)

In [6]:
urlstoscrap=[]
soup=BeautifulSoup(driver.page_source,'html.parser')
allplaces=soup.find_all('li',class_='seo-departement')
for a in allplaces:
    href=a.find('a')['href']
    urlstoscrap.append('https://annuairesante.ameli.fr'+href)

In [7]:
secondariurls=[]
for e in urlstoscrap:
    driver.get(e)
    time.sleep(2)
    soup2=BeautifulSoup(driver.page_source,'html.parser')
    try:
      first=soup2.find('ul',class_='first')
      if first:
        links=first.find_all('li')
        if links:
           for i in links:
              temphtml=i.find('a')['href']
              if temphtml:
                 secondariurls.append(temphtml)
    except AttributeError:
       print('there is no element')
    try:
      first=soup2.find('ul',class_='second')
      if first:
        links=first.find_all('li')
        if links:
           for i in links:
              temphtml=i.find('a')['href']
              if temphtml:
                 secondariurls.append(temphtml)
    except AttributeError:
       print('there is no element')
    try:
      first=soup2.find('ul',class_='third')
      if first:
        links=first.find_all('li')
        if links:
           for i in links:
              temphtml=i.find('a')['href']
              if temphtml:
                 secondariurls.append(temphtml)
    except AttributeError:
       print('there is no element')

In [8]:
name=[]
specialist_title=[]
numtel=[]
adress=[]
urlstoscrap3=[]


In [9]:
len(secondariurls)

0

In [10]:
for index,enlace in enumerate(secondariurls):
    numde=0
    driver.get('https://annuairesante.ameli.fr'+enlace)
    time.sleep(2)
    while numde==0:
      slider = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.XPATH, '//*[@id="site_inner"]/div[7]'))
                )
      soup2=BeautifulSoup(driver.page_source,'html.parser')
      targets=soup2.find_all('div',class_='item-professionnel')
      print(index)
      for target in targets:
         try:
            prename=target.find('h2').text
            preurl=target.find('a')['href']
            if preurl:
               urlstoscrap3.append('https://annuairesante.ameli.fr'+preurl)
            if prename:
               name.append(prename)
            else:
               name.append(None)
         except AttributeError:
            name.append(None)
         try:
            prespecialist_title=target.find('div',class_='item left specialite').text
            if prespecialist_title:
               specialist_title.append(prespecialist_title)
            else:
               specialist_title.append(None)
         except AttributeError:
            specialist_title.append(None)
         try:
            prenumtel=target.find('div',class_='item left tel').text
            if prenumtel:
               numtel.append(prenumtel)
            else:
               numtel.append(None)
         except AttributeError:
            try:
              prenumtel=target.find('div',class_='item right tel').text
              if prenumtel:
                  numtel.append(prenumtel)
              else:
                  numtel.append(None)
            except AttributeError:
                numtel.append(None)
         try:
            preadress=target.find('div',class_='item left adresse').text
            if preadress:
               adress.append(preadress)
            else:
               adress.append(None)
         except AttributeError:
            adress.append(None)
      if len(driver.find_elements(By.XPATH, '//img[@alt="Page suivante"]'))==0:
         numde=1
      else:
        time.sleep(1)
        next=WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//img[@alt="Page suivante"]'))
        )
      #   next=driver.find_element(By.XPATH, '//img[@alt="Page suivante"]')
        next.click()
      

In [11]:
sample=({
    'name':name,
    'specialist_title':specialist_title,
    'numtel':numtel,
    'adress':adress,
    'url':urlstoscrap3

})

In [12]:
sample=pd.DataFrame(sample)

In [13]:
sample['Services']=None

In [14]:
for indice, enlace in enumerate(sample['Services']):
    if enlace is None:
       search_url = sample['url'][index]
       driver.get(search_url)
       print(indice)
       try:
           WebDriverWait(driver, 10).until(
                   EC.presence_of_element_located((By.XPATH, '//*[@id="centresite"]/div[1]/a[1]'))
               )
           time.sleep(3)
           
       except TimeoutException :
           pass
       page=BeautifulSoup(driver.page_source,'html.parser')
   
   

       try:
           pre_servicios=page.find('div',id='popinAutocompleteContent').text.replace('\n', ' ').replace('\t', '').strip()
           if pre_servicios:
               sample['Services'][indice]=pre_servicios
       except AttributeError:
           print('There is no services')
    

In [15]:
sample=pd.DataFrame(sample).drop_duplicates()

In [16]:
sample

,name,specialist_title,numtel,adress,url,Services


In [17]:

sample.to_csv('annuairesante_with_servicesOG.csv',index=False)
sample.to_excel('annuairesante_with_servicesOG.xlsx',index=False)